In [1]:
import cv2

from utils import *
import warnings
import numpy as np
from matplotlib import pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
# Canny
c_t1 = 50
c_t2 = c_t1 * 3

model_img = cv.imread('images/0.jpg')
model_gray = cv.cvtColor(model_img, cv.COLOR_BGR2GRAY)
model_canny = cv.Canny(model_gray, c_t1, c_t2)
model_lines = cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR)
cv.imwrite("./new/cany-gray.jpg", model_lines)

True

In [3]:
h_theta = np.pi / 180
h_lines = None
h_srn = 0  # divisor for rho
h_stn = 0  # divisor for theta

# Canny
c_t1 = 50
c_t2 = c_t1 * 3

# Process model image=========================
model_img = cv.imread('images/0.jpg')
model_gray = cv.cvtColor(model_img, cv.COLOR_BGR2GRAY)
model_canny = cv.Canny(model_gray, c_t1, c_t2)
model_lines = cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR)
source_lines = cv.HoughLines(model_canny, 1.1, h_theta, 247, h_lines, h_srn, h_stn)

linesP = cv.HoughLinesP(model_canny, 1, np.pi / 180, 50, None, 50, 10)
cdstP = np.copy(cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR))

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)

cv.imwrite("model_hough_lines(cdstP).png", cdstP)

cartesian_lines = []
if source_lines is not None:
    for i in range(0, len(source_lines)):
        pt1, pt2 = add_line(source_lines, i, model_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])
        
print("cartesian_lines:", cartesian_lines)
all_intersections = junctions(cartesian_lines)
print("all_intersections:", all_intersections)
cluster_centers = kmeans_centers(all_intersections, 10)

mark_all(cluster_centers, model_lines)
cv.imwrite("model_lines.png", model_lines)

# Calculate corners
m_top, m_bot = get_model_corners(cluster_centers)
print("Cluster centers:", cluster_centers)
print("Points on top:", m_top)
print("Points on bot:", m_bot)
model_joined_topbot = m_top + m_bot

cartesian_lines: [[[-1000, 5], [999, 5]], [[-1000, 379], [999, 379]], [[-1000, 1], [999, 1]], [[5, 1000], [5, -1000]], [[348, 1000], [348, -1000]], [[695, 1000], [695, -1000]], [[0, 1000], [0, -1000]], [[122, 1000], [122, -1000]], [[579, 1000], [579, -1000]], [[126, 1000], [126, -1000]], [[576, 1000], [576, -1000]]]
all_intersections: [[5, 5], [348, 5], [695, 5], [122, 5], [579, 5], [126, 5], [576, 5], [5, 379], [348, 379], [695, 379], [122, 379], [579, 379], [126, 379], [576, 379], [5, 1], [348, 1], [695, 1], [122, 1], [579, 1], [126, 1], [576, 1]]
Cluster centers: [[124.  379. ]
 [577.5   3. ]
 [124.    3. ]
 [577.5 379. ]
 [348.    3. ]
 [348.  379. ]
 [  5.    3. ]
 [695.    3. ]
 [  5.  379. ]
 [695.  379. ]]
Points on top: [[5, 3], [695, 3]]
Points on bot: [[5, 379], [695, 379]]


In [5]:
src = cv.imread("./images/33.jpg")
dst = cv.Canny(src, 50, 200, None, 3)
frame_lines = model_lines
lines = linesP

linesP = cv.HoughLines(dst, 1,  np.pi / 180, 50, None, 0, 0)

cartesian_lines = []
if lines is not None:
    for i in range(0, len(lines)):
        pt1, pt2 = add_line(lines, i, frame_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])

junction_points = junctions(cartesian_lines)
cluster_centers = kmeans_centers(junction_points, 25)
mark_all(junction_points, frame_lines, rbg=(0, 0, 255))
n_noninf = get_noninf(cluster_centers)
top, bot = get_frame_corners(n_noninf)

mask = np.array([])
frame_joined_topbot = top + bot
mask, status = cv.findHomography(np.array(frame_joined_topbot), np.array(model_joined_topbot))
print("Mask:\n", mask)

height, width, channels = model_img.shape
warped_img = cv.warpPerspective(src, mask, (width, height))

# cv.imshow("Model", warped_img)
# cv.waitKey(0)

D/E:  -0.0
Points on top: [array([416.7892823 , 121.35665072]), array([416.7892823 , 121.35665072])]
Points on bot: [array([416.7892823 , 121.35665072]), array([416.7892823 , 121.35665072])]
Mask:
 None


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:3323: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'warpPerspective'


In [ ]:
frame_lines = model_lines
lines = linesP

cartesian_lines = []
if lines is not None:
    for i in range(0, len(lines)):
        pt1, pt2 = add_line(lines, i, frame_lines)
        cartesian_lines.append([[pt1[0],pt1[1]], [pt2[0],pt2[1]]])

# print(cartesian_lines)
# Find all junctions and perform k-means and mark them
junction_points = junctions(cartesian_lines)
# print(junction_points)
cluster_centers = kmeans_centers(junction_points, 10)
mark_all(junction_points, frame_lines)

# for num in cluster_centers:
#     print(num)
